## **一、配置訓練環境**

In [ ]:
!pip install torch torchvision torchaudio torchsummary pycocotools opencv-python opencv-contrib-python onnx onnx_tf onnx2keras
!apt-get -qq install -y build-essential git cmake libprotobuf-dev protobuf-compiler libvulkan-dev vulkan-utils libopencv-dev
!git clone https://github.com/Tencent/ncnn.git
%cd ncnn
!git checkout a03c1353193a172bfc22481195704731f75148d9
!git submodule update --init
!mkdir build
%cd build
!cmake -DCMAKE_BUILD_TYPE=Release -DNCNN_VULKAN=ON -DNCNN_SYSTEM_GLSLANG=OFF -DNCNN_BUILD_EXAMPLES=ON ..
!make
!cp /content/ncnn/build/tools/onnx/onnx2ncnn /usr/bin/onnx2ncnn

## **二、模型訓練**

掛載Google Drive雲盤並驗證是否已連接 Google Drive，如果已連接則列出你的 Google Drive 根目錄下所有的文件：

In [ ]:
# 掛載Google Drive雲盤
from google.colab import drive
drive.mount('/content/gdrive')

# 列出你的 Google Drive 根目錄下所有的文件
%ls /content/gdrive/My\ Drive/

下載模型訓練工具包，並進入工具包中：

In [ ]:
%cd /content
!git clone https://github.com/Panweitong/google-colab-tinyyolov2-model-training-script-cocopi toolkit
%cd /content/toolkit/

將 Google Drive 中的數據集文件夾拷貝到 Google Colab 的服務器上，若你的文件夾名稱不同，需將「my-project-name」換成相應文件夾的名稱：

In [4]:
your_dataset_path_on_googledrive = "my-project-name"

拷貝數據，這個過程可能會需要 1~5 分鐘，具體時間取決於文件夾大小：

In [ ]:
import shutil

shutil.copytree(
    "/content/gdrive/My Drive/" + your_dataset_path_on_googledrive + "/", 
    "/content/toolkit/data/custom/" + your_dataset_path_on_googledrive + "/"
)

檢查文件夾內的數據集是否拷貝完成，運行成功以後，指令下方會出現文件夾中的所有的內容：

In [ ]:
import os

os.listdir("/content/toolkit/data/custom/" + your_dataset_path_on_googledrive + "/")

拷貝完成以後，基於剛剛拷貝的文件夾，生成模型訓練所需要的配置文件

將會依次彈出 3 個輸入框，寫入你的項目文件夾名稱(例如my-project-name)，所要訓練的迭代次數(例如100)和使用GPU或者CPU(例如GPU)：

In [ ]:
!python generate_conf.py

進入模型訓練配置文件所在的文件夾中並開始訓練模型：

In [ ]:
import os
os.chdir("conf/" + your_dataset_path_on_googledrive)

# 開始訓練模型
!python train.py

運行到設定的迭代次數自動結束訓練或者運行到一定精度後，手動結束訓練。

## **三、模型測試**

模型測試並查看效果：

In [ ]:
# 模型測試
!bash test-train.sh

# 將測試的圖片拷貝至 Google Drive 中，查看效果
import shutil
shutil.copyfile(
    "/content/toolkit/out/" + your_dataset_path_on_googledrive + "/predictions.jpg", 
    "/content/gdrive/My Drive/" + your_dataset_path_on_googledrive + "_test.jpg"
)

## **四、導出模型**

運行模型導出工具並檢查模型是否生成：

In [ ]:
# 運行模型導出工具
!bash export.sh

# 檢查模型是否生成，如果生成了，將會有一個後綴名爲 `.bin`和一個後綴名爲 `.param` 的文件
import os
os.chdir("../../out/" + your_dataset_path_on_googledrive + "/model")
!ls ./

回到工具包文件夾主目錄下將導出的模型文件壓縮包拷貝至你的 Google Drive 中，並在彈出框內，寫下項目名稱：

In [ ]:
# 回到工具包文件夾主目錄下
%cd ../../../

# 执行拷貝程式
!python copy_to_googledrive.py

運行完成以後，前往你的 Google Drive 根目錄下檢查是否有該壓縮包，並將該壓縮包下載到電腦上。